### Item 45: Use datetime Instead of time for Local Clocks

* Coordinated Universal Time (UTC) is the standard, time-zone-independent representation of time.
* UTC works great for computers that represent time as seconds since the UNIX epoch.
* But UTC isn't ideal for humans.
* Humans reference time relative to where they're currently located.

* Time zone conversions

    * Python provides two ways of accomplishing time zone conversions.
        * The old way, using the `time` built-in module, is disastrously error prone.
        * The new way, using the `datetime`built-in module, works great with some help from the community-built package `pytz`.
        * You should be acquainted with both `time` and `datetime` to thoroughly understand why `datetime` is the best choice and `time` should be avoided.

In [ ]:
from datetime import date, time, datetime
from time import strptime, strftime

import pytz

#### Creating datetime instances

https://realpython.com/python-datetime/#creating-python-datetime-instances

#1 by passing keyword arguments

In [ ]:
dt = datetime(year=2020, month=8, day=5, hour=13, minute=14, second=31)
dt

#2 There are three ways of creating datetime instances
* `date.today()`: creates a datetime.date instance with the current local date
* `datetime.now()`: creates a datetime.datetime instance with the current local date and time
* `datetime.combine()`: combines instances of datetime.date and datetime.time into a single datetime.datetime instance.

In [ ]:
today = date.today()
today

In [ ]:
now = datetime.now()
now

In [ ]:
current_time = time(now.hour, now.minute, now.second)
print(current_time)

dt_combine = datetime.combine(today, current_time)
dt_combine

#3 Using Strings to Create Python datetime Instance

* Another way to create date instances is to use .fromisoformat(). To use this method, you provide a string with the date in the ISO 8601 format.
* You might provide a string with the year, month, and date specified.

In [ ]:
d = date.fromisoformat("2020-08-05")
d

#### strptime

* Parse the datetime string to time tuple

In [ ]:
time_format = "%Y-%m-%d %H:%M:%S"
time_str = "2020-08-05 15:45:16"

In [ ]:
time_tuple = strptime(time_str, time_format)

In [ ]:
time_tuple

#### strftime

* format a time tuple to a string (style string)

In [ ]:
d = datetime.now()
time_tuple = d.timetuple()
print(time_tuple)

In [ ]:
time_str = strftime(time_format, time_tuple)

In [ ]:
time_str

#### The time module

* The `localtime` function from the `time` built-in module lets you convert:
    * UNIX timestamp (seconds since the UNIX epoch in UTC) to a local time that matches the host computer's time zone.

In [ ]:
from time import localtime, strftime 

now = 1407694710
local_tuple = localtime(now)
print(local_tuple)

In [ ]:
time_format = "%Y-%m-%d %H:%M:%S"
time_str = strftime(time_format, local_tuple)
print(time_str)

* Conver user input local time to UNIX timestamp
    * Use strptime function to parse the time string. 
    * Then, call mktime to convert local time to UNIX timestamp

In [ ]:
from time import mktime, strptime

time_tuple = strptime(time_str, time_format)
print(time_tuple)

In [ ]:
utc_now = mktime(time_tuple)
print(utc_now)

* The `time` module fails to consistently work properly for multiple local times.
* If you must use `time`, only use it to convert between UTC and the host computer's local time.
* For all other types of conversions, use the `datetime` module.

#### The datetime module

* The second option for representing times in Python is the `datetime` class from the `datetime` built-in module.
* Like the `time` module, datetime can be used to convert from the current time in UTC to local time.

* Take the present time in UTC and conver it to my computer's local time (CST)

In [ ]:
from datetime import datetime, timezone

now = datetime.now()
now

In [ ]:
now_utc = now.replace(tzinfo=timezone.utc)
print(now_utc)

In [ ]:
now_local = now_utc.astimezone()
print(now_local)

* The `datetime` module can also easily convert a local time back to a `UNIX timestamp` in UTC.

In [ ]:
time_str

In [ ]:
now

In [ ]:
time_tuple

In [ ]:
utc_now = mktime(time_tuple)
utc_now

* Unlike the `time` module, the `datetime` module has facilities for reliable converting from one local time to another local time.
* However, `datetime` only provides the machinery for time zone operations with its `tzinfo` classes and related methods.
* What's missing are the time zone definitions besides UTC.

* `pytz` contains a full database of every time zone definition you might need.

    * https://pypi.python.org/pypi/pytz/
    
   
* To use `pytz` effectively, you should always convert local times to UTC first. [important!]
* Perform any datetime operations you need on the UTC values (such as offsetting).
* Then, convert to local times as a final step.

e.g.
* Convert an NYC flight arrival time to UTC datetime.
* Although some of these calls seem redundant, all of them are necessary when using `pytz`.

In [ ]:
arrival_nyc = "2020-08-05 23:33:24"
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)

eastern = pytz.timezone("US/Eastern")
nyc_dt = eastern.localize(nyc_dt_naive)

utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))

In [ ]:
print(utc_dt)

* Once I have a UTC datetime, I can convert it to Houston local time.

In [ ]:
central = pytz.timezone("US/Central")
houston_dt = central.normalize(utc_dt.astimezone(central))

In [ ]:
print(houston_dt)

* Just as easily, I can convert it to the local time in Nepal.

In [ ]:
nepal = pytz.timezone("Asia/Katmandu")
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))

In [ ]:
print(nepal_dt)

* With `datetime` and `pytz`, these conversions are consistent across all environments regardless of what operating system the host computer is running.

### Things to Remember

* Avoid using the `time` module for translating between different time zones.
* Use the `datetime` built-in module along with the `pytz` module to reliably convert between times in different time zones.
* Always represent time in UTC and do conversions to local time as the final step before presentation.